# HBA PA3

French version: https://github.com/Daniel-R-Armstrong/French-Wiki-2500-Pretrained-SentencePiece-LM/blob/master/fr_spm.ipynb
https://forums.fast.ai/t/sentencepiece/53010/50

# Turkish ULMFiT from scratch

In [ ]:
# #hide
# !pip install -Uqq fastbook
# import fastbook
# fastbook.setup_book()

In [ ]:
# import sys
# !{sys.executable} -m pip install fastai==1.0.61 #This right here

In [ ]:
# import sys
# !{sys.executable} -m pip install fastcore==1. #This right here

In [ ]:
# #fastai v1 API: https://fastai1.fast.ai/overview.html
# from fastai.basics import *

In [ ]:
# from fastai.utils import *
# show_install()

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [2]:
print ('hello')

hello


In [3]:
from fastai.imports import *

In [4]:
data_path = Path('/home/ec2-user/.fastai/data')

In [5]:
bs=128
torch.cuda.set_device(2)
# data_path = Config.data_path()
# data_path = '/home/ec2-user/.fastai/data'

lang = 'tr'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)

In [6]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
lm_fns = [mdl_path/f'{lang}_wt', mdl_path/f'{lang}_wt_vocab']

In [7]:
path

PosixPath('/home/ec2-user/.fastai/data/trwiki')

In [8]:
lang

'tr'

## Turkish wikipedia model

In [8]:
# #HBA
# import sys
# !{sys.executable} -m pip install wikiextractor

In [9]:
# !pip install wikiextractor

In [9]:
from nlputils import split_wiki,get_wiki

## Uncompress wiki file from cli

Source: https://github.com/attardi/wikiextractor

1. Do `python setup.py install`
1. Double check with: `which wikiextractor`

`
base) [ec2-user@ip-172-16-69-8 trwiki]$ which wikiextractor
~/anaconda3/bin/wikiextractor
`

1. Uncrompress

`
(base) [ec2-user@ip-172-16-69-8 trwiki]$ python -m wikiextractor.WikiExtractor trwiki-latest-pages-articles.xml
INFO: 2832129   Mucize Dünyası: New York – Birleşik Kahramanlar
INFO: 2832111   Kraliyet Mühendislik Akademisi üyesi kadınlar listesi
INFO: 2832131   Ilana Rovina
INFO: 2832155   Artur Taymazov
INFO: 2832130   Enka (kadın voleybol takımı) 1992-93 sezonu
INFO: Finished 31-process extraction of 377951 articles in 123.1s (3071.5 art/s)
INFO: total of page: 732729, total of articl page: 377951; total of used articl page: 377951
(base) [ec2-user@ip-172-16-69-8 trwiki]$ which wikiextractor
~/anaconda3/bin/wikiextractor
`
    


In [10]:
get_wiki(path,lang)

downloading...
unzipping...
extracting... akirmak


In [11]:
# from nlputils import split_wiki,get_wiki

# get_wiki(path,lang)
!head -n4 {path}/{name}

<doc id="10" url="https://tr.wikipedia.org/wiki?curid=10" title="Cengiz Han">
Cengiz Han

Cengiz Han (doğum Temuçin, – 18 Ağustos 1227) ölümünden sonra tarihin en büyük yüzölçümüne sahip imparatorluğu haline gelmiş Moğol İmparatorluğu'nun kurucusu ve ilk Kağanı ve İmparatoru olan Moğol komutan ve hükümdar. 13. Yüzyılın başında Orta Asya'daki tüm göçebe bozkır kavimlerini birleştirerek bir ulus hâline getirdi ve o ulusu "Moğol" siyasi kimliği çatısı altında topladı.


In [12]:
dest = split_wiki(path,lang)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


Turkish is an [Agglutinative_language](https://en.wikipedia.org/wiki/Agglutinative_language) so it needs special care!

![Turkish morphemes example](images/turkish.jpg)

# HBA fix

1. Open via: `vi ~/SageMaker/.env/fastai/lib/python3.6/site-packages/fastai/text/data.py`
1. see: https://forums.fast.ai/t/multifit-runtime-error-permission-denied/72874/5
1. See: https://github.com/fastai/fastai/issues/2572 

In [ ]:
data = (TextList.from_folder(dest, processor=[OpenFileProcessor(), SPProcessor()])
        .split_by_rand_pct(0.1, seed=42)
        .label_for_lm()
        .databunch(bs=bs, num_workers=1))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/.env/fastai/lib/python3.6/multiprocessing/queues.py", line 234, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/home/ec2-user/SageMaker/.env/fastai/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
_pickle.PicklingError: Can't pickle <class 'fastai.text.data.SPProcessor'>: it's not the same object as fastai.text.data.SPProcessor
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/.env/fastai/lib/python3.6/multiprocessing/queues.py", line 234, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/home/ec2-user/SageMaker/.env/fastai/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
_pickle.PicklingError: Can't pickle <class 'fastai.text.data.SPProcessor'>: it's not the same object as fastai.text.data.SPProcessor
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/.env/fastai/lib/python3.6/mu

In [ ]:
data = load_data(dest, f'{lang}_databunch', bs=bs)

In [ ]:
data.show_batch()

In [ ]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.1, wd=0.1, pretrained=False).to_fp16()

In [ ]:
lr = 3e-3
lr *= bs/48  # Scale learning rate by batch size

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

In [ ]:
learn.to_fp32().save(lm_fns[0], with_opt=False)
learn.data.vocab.save(lm_fns[1].with_suffix('.pkl'))

## Turkish sentiment analysis

https://www.win.tue.nl/~mpechen/projects/smm/

### Language model

In [ ]:
path_clas = path/'movies'
path_clas.ls()

In [ ]:
pos = (path_clas/'tr_polarity.pos').open(encoding='iso-8859-9').readlines()
pos_df = pd.DataFrame({'text':pos})
pos_df['pos'] = 1
pos_df.head()

In [ ]:
neg = (path_clas/'tr_polarity.neg').open(encoding='iso-8859-9').readlines()
neg_df = pd.DataFrame({'text':neg})
neg_df['pos'] = 0
neg_df.head()

In [ ]:
df = pd.concat([pos_df,neg_df], sort=False)

In [ ]:
data_lm = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

data_lm.save(f'{lang}_clas_databunch')

In [ ]:
data_lm = load_data(path_clas, f'{lang}_clas_databunch', bs=bs)

In [ ]:
data_lm.show_batch()

In [ ]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0, wd=0.1)

In [ ]:
lr = 1e-3
lr *= bs/48

In [ ]:
learn_lm.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

In [ ]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(5, slice(lr/10,lr*10), moms=(0.8,0.7))

In [ ]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

### Classifier

In [ ]:
data_clas = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='pos')
    .databunch(bs=bs, num_workers=1))

In [ ]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False, wd=0.1).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [ ]:
lr=2e-2
lr *= bs/48

In [ ]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

In [ ]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

In [ ]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

In [ ]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

In [ ]:
learn_c.unfreeze()
learn_c.fit_one_cycle(4, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

Accuracy in Gezici (2018), *Sentiment Analysis in Turkish* is: `75.16%`.

In [ ]:
learn_c.save(f'{lang}clas')

## fin